In [ ]:
tifffile.memmap()

In [114]:
#create an empty TZYX ImageJ hyperstack
hyperstack_path='/scratch/zimmer/Ulises_temp/ImageJ_Stack16bit_v2.tif'
hyperstack = tiff.memmap(
    hyperstack_path,
    shape=(10000, 20, 700, 900),
    dtype='uint16',
    imagej=True,
    metadata={'axes': 'TZYX'},
)

print(type(hyperstack), hyperstack.shape)


/users/ulises.rey/anaconda3/envs/DLC-GPU/lib/python3.6/site-packages/tifffile/tifffile.py:2016: UserWarning: truncating ImageJ file
  warnings.warn('truncating ImageJ file', UserWarning)


<class 'numpy.memmap'> (10000, 20, 700, 900)


In [ ]:

# write an image to the numpy.memmap array
hyperstack[t_index, z_index] = image
hyperstack.flush()

In [115]:
path='/scratch/zimmer/Ulises_temp/wbfm_2021_04_01/2021-04-01_09-34-04_worm1-channel-0-pco_camera1/'

In [ ]:
c=0
z_index=0
t_index=0
for file in natsorted(os.listdir(path)):
    if file.endswith('ome.tif'):
        print(os.path.join(path,file))
        with tiff.TiffFile(os.path.join(path,file), multifile=False) as tif:
            for idx,page in enumerate(tif.pages):
                img = page.asarray()
                hyperstack[t_index, z_index] = img
                hyperstack.flush()
                c=c+1
                z_index=z_index+1
                if z_index==20:
                    z_index=0
                    t_index=t_index+1

/scratch/zimmer/Ulises_temp/wbfm_2021_04_01/2021-04-01_09-34-04_worm1-channel-0-pco_camera1/2021-04-01_09-34-04_worm1-channel-0-pco_camera1_MMStack.ome.tif
/scratch/zimmer/Ulises_temp/wbfm_2021_04_01/2021-04-01_09-34-04_worm1-channel-0-pco_camera1/2021-04-01_09-34-04_worm1-channel-0-pco_camera1_MMStack_1.ome.tif
/scratch/zimmer/Ulises_temp/wbfm_2021_04_01/2021-04-01_09-34-04_worm1-channel-0-pco_camera1/2021-04-01_09-34-04_worm1-channel-0-pco_camera1_MMStack_2.ome.tif
/scratch/zimmer/Ulises_temp/wbfm_2021_04_01/2021-04-01_09-34-04_worm1-channel-0-pco_camera1/2021-04-01_09-34-04_worm1-channel-0-pco_camera1_MMStack_3.ome.tif
/scratch/zimmer/Ulises_temp/wbfm_2021_04_01/2021-04-01_09-34-04_worm1-channel-0-pco_camera1/2021-04-01_09-34-04_worm1-channel-0-pco_camera1_MMStack_4.ome.tif
/scratch/zimmer/Ulises_temp/wbfm_2021_04_01/2021-04-01_09-34-04_worm1-channel-0-pco_camera1/2021-04-01_09-34-04_worm1-channel-0-pco_camera1_MMStack_5.ome.tif
/scratch/zimmer/Ulises_temp/wbfm_2021_04_01/2021-04-01

In [31]:
input_filepath='/scratch/zimmer/Ulises_temp/ImageJ_Stack16bit.tif'


In [ ]:
tif.series[0].pages.shape

In [ ]:
with tiff.TiffFile(input_filepath) as tif:
    print('looping through reader object..')
    for idx, page in enumerate(tif.pages):
        img=page.asarray()
        plt.figure(figsize=(10,10), dpi=100)
        plt.imshow(img, vmin=100, vmax=1000)
        plt.title(idx)
        plt.show()
        if idx==40:
            print('break')
            break

In [87]:
tif.series[0].pages[0].shape

(700, 900)

In [33]:
for volume in range(0, hyperstack.shape[0]):
    if volume%100==0:
        print('volume',volume)

        for frame in range(0, hyperstack.shape[1]):

            print('frame',frame)
    if volume>1000: break
    

volume 0
frame 0
frame 1
frame 2
frame 3
frame 4
frame 5
frame 6
frame 7
frame 8
frame 9
frame 10
frame 11
frame 12
frame 13
frame 14
frame 15
frame 16
frame 17
frame 18
frame 19
frame 20
frame 21
frame 22
frame 23
frame 24
frame 25
frame 26
frame 27
frame 28
frame 29
volume 100
frame 0
frame 1
frame 2
frame 3
frame 4
frame 5
frame 6
frame 7
frame 8
frame 9
frame 10
frame 11
frame 12
frame 13
frame 14
frame 15
frame 16
frame 17
frame 18
frame 19
frame 20
frame 21
frame 22
frame 23
frame 24
frame 25
frame 26
frame 27
frame 28
frame 29
volume 200
frame 0
frame 1
frame 2
frame 3
frame 4
frame 5
frame 6
frame 7
frame 8
frame 9
frame 10
frame 11
frame 12
frame 13
frame 14
frame 15
frame 16
frame 17
frame 18
frame 19
frame 20
frame 21
frame 22
frame 23
frame 24
frame 25
frame 26
frame 27
frame 28
frame 29
volume 300
frame 0
frame 1
frame 2
frame 3
frame 4
frame 5
frame 6
frame 7
frame 8
frame 9
frame 10
frame 11
frame 12
frame 13
frame 14
frame 15
frame 16
frame 17
frame 18
frame 19
frame 20

In [ ]:
#ometiff2bigtiff


In [14]:
path='/scratch/zimmer/Ulises_temp/wbfm_2021_04_01/2021-04-01_09-34-04_worm1-channel-0-pco_camera1/'

In [2]:
c=0
for file in natsorted(os.listdir(path)):
#     print(f'list is {os.listdir(path)}')
#     print(os.path.join(path,file))
    if file.endswith('ome.tif'):
        print(os.path.join(path,file))
        with tiff.TiffFile(os.path.join(path,file), multifile=False) as tif:
            for idx,page in enumerate(tif.series[0].pages):
                c=c+1
                if idx%100==0:print(idx)
            #print('entered writing')
#             hyperstack = tif.asarray()
#             break
print('total pages is :', c)

NameError: name 'natsorted' is not defined